## Opening new Gym in Pune ,India.##
## A Business owner wants to open a Gym in Pune,India.He is worried in which locations he should open a Gym so that he gains him maximum profit. We are going to help him find some locations where he can open a new Gym which will benifits his business.  

    1.Build a dataframe of neighborhoods in Pune, India by web scraping the data from Wikipedia page
    2.Get the geographical coordinates of the neighborhoods
    3.Obtain the venue data for the neighborhoods from Foursquare API
    4.Explore and cluster the neighborhoods
    5.Select the best cluster to open a new Gym

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
#!pip install geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates
#!pip install folium
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
#Ignore ssl certificates
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
#URL of wikipedia which contains neighbourhour data of Pune
url="https://en.wikipedia.org/wiki/Template:Neighbourhoods_of_Pune"
url

'https://en.wikipedia.org/wiki/Template:Neighbourhoods_of_Pune'

In [4]:
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Template:Neighbourhoods of Pune - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XqlZPwpAMMQAAlAnWN0AAACV","wgCSPNonce":!1,"wgCanonicalNamespace":"Template","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":10,"wgPageName":"Template:Neighbourhoods_of_Pune","wgTitle":"Neighbourhoods of Pune","wgCurRevisionId":941587668,"wgRevisionId":941587668,"wgArticleId":26322286,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Neighbourhoods in Pune"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRelevantPageName":"

In [5]:
#Making a list of neigbourhood
neighborhoodList = []
for i in soup.find_all("td",{'class':'navbox-list navbox-odd hlist'})[0].find_all("a"):
    neighborhoodList.append(i.text)
neighborhoodList    

['Aundh',
 'Balewadi',
 'Baner',
 'Bavdhan',
 'Bhosari',
 'Camp',
 'Chakan',
 'Charholi Budruk',
 'Chinchwad',
 'Dapodi',
 'Dehu Road Cantonment',
 'Dehu',
 'Dhayari',
 'Hadapsar',
 'Hinjawadi',
 'Kalewadi',
 'Kalyani Nagar',
 'Khadki cantonment',
 'Kharadi',
 'Koregaon Park',
 'Kothrud',
 'Manjri',
 'Nanded',
 'Pashan',
 'Paud',
 'Pimple Gurav',
 'Pimple Saudagar',
 'Pimpri',
 'Rahatani',
 'Ravet',
 'Sangvi',
 'Sant Tukaram Nagar',
 'Shivajinagar',
 'Sus',
 'Swargate',
 'Thergaon',
 'Viman Nagar',
 'Wakad',
 'Wanowrie',
 'Warje',
 'Yerawada']

In [6]:
#Making Dataframe 
neighbourhood = pd.DataFrame({"Neighborhood": neighborhoodList})

neighbourhood.head()

,Neighborhood
0,Aundh
1,Balewadi
2,Baner
3,Bavdhan
4,Bhosari


In [7]:
neighbourhood.shape

(41, 1)

## We have total 41 neighbourhood in Pune##

In [8]:
#Function to get the coordinates of the location
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Pune, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
coords = [ get_latlng(neighbor) for neighbor in neighbourhood["Neighborhood"].tolist() ]
coords

[[18.563450000000046, 73.81227000000007],
 [18.576020000000028, 73.77983000000006],
 [18.548200000000065, 73.77316000000008],
 [18.517541553892233, 73.77852901740273],
 [18.638730000000066, 73.83748000000008],
 [18.524590000000046, 73.87878000000006],
 [18.734150000000056, 73.85856000000007],
 [18.640720000000044, 73.90397000000007],
 [18.647430000000043, 73.80002000000007],
 [18.494410000000073, 74.39857000000006],
 [18.679160000000024, 73.73255000000006],
 [18.720070000000078, 73.76577000000003],
 [18.447020000000066, 73.80757000000006],
 [18.502530000000036, 73.92706000000004],
 [18.591420000000028, 73.73895000000005],
 [18.394240000000025, 74.05088000000006],
 [18.54645000000005, 73.90067000000005],
 [18.569210000000055, 73.84566000000007],
 [18.544620000000066, 73.93922000000003],
 [18.535330000000044, 73.89382000000006],
 [18.505170000000078, 73.80245000000008],
 [18.481940497085816, 73.86561799417163],
 [18.456420000000037, 73.79200000000003],
 [18.536740000000066, 73.7929000000

In [10]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df_coords.head()

,Latitude,Longitude
0,18.563450,73.812270
1,18.576020,73.779830
2,18.548200,73.773160
3,18.517542,73.778529
4,18.638730,73.837480


In [11]:
#Neighbourhood with Latitude and Longitute
neighbourhood['Latitude']=df_coords['Latitude']
neighbourhood['Longitude']=df_coords['Longitude']
print(neighbourhood.shape)
neighbourhood

(41, 3)


,Neighborhood,Latitude,Longitude
0,Aundh,18.563450,73.812270
1,Balewadi,18.576020,73.779830
2,Baner,18.548200,73.773160
3,Bavdhan,18.517542,73.778529
4,Bhosari,18.638730,73.837480
5,Camp,18.524590,73.878780
6,Chakan,18.734150,73.858560
7,Charholi Budruk,18.640720,73.903970
8,Chinchwad,18.647430,73.800020
9,Dapodi,18.494410,74.398570


In [12]:
#Get the location coordinates of Pune,India
address = 'Pune, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Pune, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Pune, India 18.521428, 73.8544541.


In [13]:
#Map of Pune and its neighbourhood
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(neighbourhood['Latitude'], neighbourhood['Longitude'], neighbourhood['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [14]:
#Foursquare Credentials
CLIENT_ID = 'WJLEWTB3NOANHL5A1J4SJ3DTP3OAKPM4PI0CT5SP4A4IHHG1' # your Foursquare ID
CLIENT_SECRET = 'VLJWVMYTEQCTL51ERQM3RTSO4Z0TI4GYOGMIOEDM23I2IZPX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: WJLEWTB3NOANHL5A1J4SJ3DTP3OAKPM4PI0CT5SP4A4IHHG1
CLIENT_SECRET:VLJWVMYTEQCTL51ERQM3RTSO4Z0TI4GYOGMIOEDM23I2IZPX


We are going to explore each and every neighbourhood in Pune.

In [15]:
#Obtain the venue data for the neighborhoods from Foursquare API
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    LIMIT=200
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
Pune_venues = getNearbyVenues(names=neighbourhood['Neighborhood'],
                                   latitudes=neighbourhood['Latitude'],
                                   longitudes=neighbourhood['Longitude'])

Aundh
Balewadi
Baner
Bavdhan
Bhosari
Camp
Chakan
Charholi Budruk
Chinchwad
Dapodi
Dehu Road Cantonment
Dehu
Dhayari
Hadapsar
Hinjawadi
Kalewadi
Kalyani Nagar
Khadki cantonment
Kharadi
Koregaon Park
Kothrud
Manjri
Nanded
Pashan
Paud
Pimple Gurav
Pimple Saudagar
Pimpri
Rahatani
Ravet
Sangvi
Sant Tukaram Nagar
Shivajinagar
Sus
Swargate
Thergaon
Viman Nagar
Wakad
Wanowrie
Warje
Yerawada


In [17]:
print(Pune_venues.shape)
Pune_venues.head()

(3382, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aundh,18.56345,73.81227,Crosswords,18.556177,73.809131,Bookstore
1,Aundh,18.56345,73.81227,Mithas,18.554635,73.809369,Dessert Shop
2,Aundh,18.56345,73.81227,Yolkshire,18.553576,73.806888,English Restaurant
3,Aundh,18.56345,73.81227,Pagdandi,18.551972,73.793091,Coffee Shop
4,Aundh,18.56345,73.81227,Lost The Plot,18.554417,73.804808,Lounge


We have total of 3382 Venue in Pune.

In [18]:
Pune_venues['Venue Category'].replace({'Gym / Fitness Center':'Gym'},inplace=True)

In [19]:
Pune_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aundh,18.56345,73.81227,Crosswords,18.556177,73.809131,Bookstore
1,Aundh,18.56345,73.81227,Mithas,18.554635,73.809369,Dessert Shop
2,Aundh,18.56345,73.81227,Yolkshire,18.553576,73.806888,English Restaurant
3,Aundh,18.56345,73.81227,Pagdandi,18.551972,73.793091,Coffee Shop
4,Aundh,18.56345,73.81227,Lost The Plot,18.554417,73.804808,Lounge


In [20]:
#Counting Venue Category in each neighbourhood
Pune_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aundh,100,100,100,100,100,100
Balewadi,100,100,100,100,100,100
Baner,100,100,100,100,100,100
Bavdhan,100,100,100,100,100,100
Bhosari,73,73,73,73,73,73
Camp,100,100,100,100,100,100
Chakan,6,6,6,6,6,6
Charholi Budruk,51,51,51,51,51,51
Chinchwad,100,100,100,100,100,100


In [21]:
print('There are {} uniques categories.'.format(len(Pune_venues['Venue Category'].unique())))

There are 119 uniques categories.


In [22]:
Pune_venues['Venue Category'].unique()

array(['Bookstore', 'Dessert Shop', 'English Restaurant', 'Coffee Shop',
       'Lounge', 'Ice Cream Shop', 'South Indian Restaurant',
       'Shopping Mall', 'Hotel', 'Asian Restaurant', 'Italian Restaurant',
       'Indian Restaurant', 'Bakery', 'Malay Restaurant', 'BBQ Joint',
       'Multiplex', 'Bistro', 'Breakfast Spot', 'Donut Shop',
       'Chinese Restaurant', 'Café', 'Nightclub', 'Juice Bar',
       'Other Great Outdoors', 'Fast Food Restaurant', 'Brewery',
       'Vegetarian / Vegan Restaurant', 'Pub', 'Gym', 'Tea Room',
       'Sandwich Place', 'Burger Joint', 'Snack Place',
       'Seafood Restaurant', 'History Museum', 'Garden', 'Golf Course',
       'Hotel Bar', 'Trail', 'Food Truck', 'Bar', 'Brazilian Restaurant',
       'IT Services', 'French Restaurant', 'Mexican Restaurant',
       'American Restaurant', 'Maharashtrian Restaurant', 'Train Station',
       'Pizza Place', 'Stadium', 'Historic Site', 'Mountain', 'Lake',
       'Dance Studio', 'Chocolate Shop', 'Middle E

# Analyze Each Neighborhood

In [23]:
# one hot encoding
Pune_onehot = pd.get_dummies(Pune_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Pune_onehot['Neighborhood'] = Pune_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Pune_onehot.columns[-1]] + list(Pune_onehot.columns[:-1])
Pune_onehot = Pune_onehot[fixed_columns]

print(Pune_onehot.shape)
Pune_onehot.head()


(3382, 120)


,Neighborhood,ATM,Accessories Store,Airport Service,American Restaurant,Arcade,Asian Restaurant,BBQ Joint,Bakery,Bar,...,Theme Park,Toll Booth,Toll Plaza,Tourist Information Center,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Village,Zoo
0,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
Pune_grouped = Pune_onehot.groupby('Neighborhood').mean().reset_index()
Pune_grouped.head()

,Neighborhood,ATM,Accessories Store,Airport Service,American Restaurant,Arcade,Asian Restaurant,BBQ Joint,Bakery,Bar,...,Theme Park,Toll Booth,Toll Plaza,Tourist Information Center,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Village,Zoo
0,Aundh,0.0,0.0,0.000000,0.000000,0.0,0.020000,0.020000,0.050000,0.01,...,0.0,0.0,0.0,0.0,0.01,0.000000,0.0,0.01,0.0,0.0
1,Balewadi,0.0,0.0,0.000000,0.010000,0.0,0.010000,0.030000,0.020000,0.01,...,0.0,0.0,0.0,0.0,0.00,0.010000,0.0,0.03,0.0,0.0
2,Baner,0.0,0.0,0.000000,0.010000,0.0,0.010000,0.030000,0.010000,0.01,...,0.0,0.0,0.0,0.0,0.01,0.000000,0.0,0.04,0.0,0.0
3,Bavdhan,0.0,0.0,0.000000,0.000000,0.0,0.010000,0.030000,0.010000,0.01,...,0.0,0.0,0.0,0.0,0.01,0.000000,0.0,0.04,0.0,0.0
4,Bhosari,0.0,0.0,0.013699,0.013699,0.0,0.013699,0.013699,0.041096,0.00,...,0.0,0.0,0.0,0.0,0.00,0.013699,0.0,0.00,0.0,0.0


In [25]:
len(Pune_grouped[Pune_grouped["Gym"] > 0])

32

In [26]:
Pune_gym = Pune_grouped[["Neighborhood","Gym"]]

In [27]:
Pune_gym.head()

,Neighborhood,Gym
0,Aundh,0.040000
1,Balewadi,0.050000
2,Baner,0.040000
3,Bavdhan,0.040000
4,Bhosari,0.054795


# Cluster Neighborhoods

In [28]:
# set number of clusters
kclusters = 3

Pune_clustering = Pune_gym.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Pune_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 2, 1, 1, 2, 1], dtype=int32)

In [29]:
Pune_merged = Pune_gym.copy()

# add clustering labels
Pune_merged["Cluster Labels"] = kmeans.labels_
Pune_merged

,Neighborhood,Gym,Cluster Labels
0,Aundh,0.040000,0
1,Balewadi,0.050000,0
2,Baner,0.040000,0
3,Bavdhan,0.040000,0
4,Bhosari,0.054795,0
5,Camp,0.030000,2
6,Chakan,0.000000,1
7,Charholi Budruk,0.000000,1
8,Chinchwad,0.020000,2
9,Dapodi,0.000000,1


In [30]:
Pune_merged = Pune_merged.join(neighbourhood.set_index("Neighborhood"), on="Neighborhood")

print(Pune_merged.shape)
Pune_merged.head() # check the last columns!

(41, 5)


,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
0,Aundh,0.040000,0,18.563450,73.812270
1,Balewadi,0.050000,0,18.576020,73.779830
2,Baner,0.040000,0,18.548200,73.773160
3,Bavdhan,0.040000,0,18.517542,73.778529
4,Bhosari,0.054795,0,18.638730,73.837480


In [32]:
print(Pune_merged.shape)
# sort the results by Cluster Labels
Pune_merged.sort_values(["Cluster Labels"], inplace=True)
Pune_merged

(41, 5)


,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
0,Aundh,0.040000,0,18.563450,73.812270
35,Thergaon,0.040000,0,18.613820,73.773580
34,Swargate,0.050000,0,18.499310,73.858400
33,Sus,0.040000,0,18.546700,73.751130
32,Shivajinagar,0.040000,0,18.537230,73.838080
25,Pimple Gurav,0.050000,0,18.589000,73.818150
23,Pashan,0.050000,0,18.536740,73.792900
22,Nanded,0.040000,0,18.456420,73.792000
21,Manjri,0.050000,0,18.481940,73.865618
39,Warje,0.050000,0,18.472110,73.802130


In [36]:
# create map
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Pune_merged['Latitude'], Pune_merged['Longitude'], Pune_merged['Neighborhood'], Pune_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


# Cluster 1

In [40]:
Pune_merged.loc[Pune_merged['Cluster Labels'] == 0]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
0,Aundh,0.040000,0,18.563450,73.812270
35,Thergaon,0.040000,0,18.613820,73.773580
34,Swargate,0.050000,0,18.499310,73.858400
33,Sus,0.040000,0,18.546700,73.751130
32,Shivajinagar,0.040000,0,18.537230,73.838080
25,Pimple Gurav,0.050000,0,18.589000,73.818150
23,Pashan,0.050000,0,18.536740,73.792900
22,Nanded,0.040000,0,18.456420,73.792000
21,Manjri,0.050000,0,18.481940,73.865618
39,Warje,0.050000,0,18.472110,73.802130


# Cluster 2

In [38]:
Pune_merged.loc[Pune_merged['Cluster Labels'] == 1]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
10,Dehu,0.000000,1,18.72007,73.76577
36,Viman Nagar,0.000000,1,18.56520,73.91391
31,Sant Tukaram Nagar,0.010989,1,18.58277,73.98596
30,Sangvi,0.000000,1,18.16756,73.68701
27,Pimpri,0.000000,1,18.34710,74.20510
24,Paud,0.000000,1,18.52411,73.61576
6,Chakan,0.000000,1,18.73415,73.85856
40,Yerawada,0.010000,1,18.56906,73.88159
9,Dapodi,0.000000,1,18.49441,74.39857
13,Hadapsar,0.010000,1,18.50253,73.92706


# Cluster 3

In [39]:
Pune_merged.loc[Pune_merged['Cluster Labels'] == 2]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
8,Chinchwad,0.020000,2,18.64743,73.80002
38,Wanowrie,0.030000,2,18.49538,73.90009
37,Wakad,0.020000,2,18.60540,73.75557
14,Hinjawadi,0.030000,2,18.59142,73.73895
17,Khadki cantonment,0.030000,2,18.56921,73.84566
19,Koregaon Park,0.030000,2,18.53533,73.89382
29,Ravet,0.020408,2,18.64513,73.73638
28,Rahatani,0.020000,2,18.59925,73.78636
11,Dehu Road Cantonment,0.025316,2,18.67916,73.73255
5,Camp,0.030000,2,18.52459,73.87878



*Observations* :

Most of the Gyms are concentrated in the central area of Pune city, with the highest number in cluster 2 and moderate number in cluster 0. On the other hand, cluster 1 has very low number to totally no Gym in the neighborhoods. This represents a great opportunity and high potential areas to open new Gym as there is very little to no competition from existing Gyms. Meanwhile, Gyms in cluster 2 are likely suffering from intense competition due to high concentration of Gyms. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few Gyms. Therefore, this project recommends property developers to capitalize on these findings to open new Gym in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new Gyms in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of Gyms and suffering from intense competition.
